<a href="https://colab.research.google.com/github/cianc/AoC2023/blob/main/day12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
TEST = False #@param {type: "boolean"}
TEST_INPUT1 = 'day12_test1.txt'
TEST_INPUT2 = 'day12_test1.txt'
INPUT = 'day12_input1.txt'
PART=1 #@param {type: "integer"}


In [14]:
!pip install -Uqq ipdb
!pip install -Uqq sympy


In [78]:
import copy
import ipdb
import itertools
import re
import time

from sympy.utilities.iterables import partitions

class Row():
  def __init__(self, springs: str, damage_counts: list[int]):
    self.springs = springs
    self.damage_counts = damage_counts
    self.blocks = self._get_blocks()

  def __repr__(self):
    return f'{self.springs} {",".join(self.damage_counts)}'

  def _get_blocks(self):
    block_regex = r'[\?\#]+'
    return [block for block in re.findall(block_regex, self.springs)]


def get_input() -> list[str]:
  if PART == 1:
    input = TEST_INPUT1 if TEST else INPUT
  else:
    input = TEST_INPUT2 if TEST else INPUT

  with open(input, 'r') as f:
    rows = f.read().splitlines()
  return rows


def parse_map(input: list[str]) -> list[Row]:
  map = []
  row_regex = r'^([\?\#\.]+) (([0-9]+,?)+)$'
  for input_row in input:
    m = re.match(row_regex, input_row)
    map.append(Row(m.group(1), [int(c) for c in m.group(2).split(',')]))
  return map


def block_combinations(block: str) -> list[str]:
  block_combinations = []

  mystery_indexes = [m.start() for m in re.finditer(r'\?', block)]
  mystery_index_combinations = []
  for i in range(len(mystery_indexes)+1):
    mystery_index_combinations.extend(list(itertools.combinations(mystery_indexes, i)))

  for mic in mystery_index_combinations:
    temp_block = copy.copy(block).replace('?', '.')
    for i in mic:
      temp_block = temp_block[:i] + '#' + temp_block[i+1:]
    block_combinations.append(temp_block)
  return block_combinations

def find_all_possible_matches(blocks: list[str], damage_counts: list[int], prev_matches: list[str] = []) -> int:
  # Base case
  print('===============')
  print(f'blocks: {blocks}')
  print(f'damage_counts: {damage_counts}')
  if not damage_counts:
    return 0

  damage_string = '#' * damage_counts[0]

  full_match_count = 0
  #ipdb.set_trace()
  for block in blocks:
    print(f'block: {block}')
    combinations = block_combinations(block)
    for i in range(len(combinations)):
      combination = combinations[i]
      #ipdb.set_trace()
      print(f'combination: {combination}')
      print(f'{i} -- damage_string: {damage_string}')
      match_index = combination.index(damage_string) if damage_string in combination else -1
      if match_index >= 0:
        print(f'partial match: {combination} - {damage_string}')
        prev_matches.append((combination, damage_string))
        if len(damage_counts) == 1:
          print(f'!!MATCH!! - {prev_matches}')
          full_match_count += 1
        else:
          subblock = block[match_index+len(damage_string):]
          subblock = [subblock] if subblock else []
          blocks_remainder = blocks[1:]
          new_blocks  = subblock + blocks_remainder
          print(f'new_blocks: {new_blocks}')
          if new_blocks:
            print(f'recurse1')
            full_match_count += find_all_possible_matches(
              [block[match_index+len(damage_string):]] + blocks[1:], damage_counts[1:], prev_matches)
      else:
        print(f'no match')
        if len(blocks) > 1:
          print(f'recurse2')
          full_match_count += find_all_possible_matches(blocks[1:], copy.copy(damage_counts))
  return full_match_count

def possible_damage_strings(damage_counts: list[int], row_length: int) -> list[str]:
  gap_count = len(damage_counts) -1
  extra_spacers = row_length - (sum(damage_counts) + gap_count)

  spacer_placements = []
  spacer_placement_count = 2 + gap_count
  ps = partitions(extra_spacers, m=spacer_placement_count)
  for p in ps:
    local_answer = []
    for val, count in p.items():
      local_answer.extend([val for i in range(count)])
    local_answer = local_answer + [0] * (spacer_placement_count - len(local_answer))
    local_answer = list(set(itertools.permutations(local_answer, spacer_placement_count)))
    spacer_placements.extend(local_answer)

  answers = []
  for sp in spacer_placements:
    answer = ''
    for i, dc in enumerate(damage_counts):
      if sp[i] > 0:
        answer += ('.' * sp[i])
      answer += ('#' * dc) + '.'
    answer = answer[:-1]
    if sp[-1] > 0:
      answer += ('.' * sp[-1])
    answers.append(answer)

  return answers

def get_matches(possible_damage_strings: list[str], row: str) -> int:
  match_count = 0
  for pds in possible_damage_strings:
    pairs = zip(pds, row)
    if all([(p == r or r == '?') for p, r in pairs]):
      match_count += 1
  return match_count

start_time = time.time()
input = get_input()
map = parse_map(input)
answer = 0
for row in map:
  possible_strings = possible_damage_strings(row.damage_counts, len(row.springs))
  local_answer = get_matches(possible_strings, row.springs)
  answer += local_answer
print(f'answer: {answer}')
end_time = time.time()
print(f'run time: {end_time - start_time} (s)')

answer: 7110
run time: 0.6884071826934814 (s)
